In [1]:
import pandas as pd
import numpy as np

#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import db_utils
from helpers import settings

bucket = S3BucketUtils()
################

date_of_analysis = input('Date of analysis: ')
date_dir = date_of_analysis.replace('-', '_')

Date of analysis: 2021-12-01


In [2]:
data_sets = ['data_tv_ALL_spots_with_CB_wo_151617.csv',\
             'data_tv_ALL_spots_wo_CB_wo_151617.csv',\
             'data_tv_CAN_CANCEL_spots_with_CB_wo_151617.csv',\
             'data_tv_CAN_CANCEL_spots_wo_CB_wo_151617.csv']

In [3]:
def get_months_with_properly_using_oos_perc(spot):
    num_of_months_with_oos = len(spot[spot['had_online_orders_last_month']==1])
    num_of_months_with_properly_using_oos = len(spot[spot['had_properly_used_online_orders_inquiries_last_month']==1])
    if num_of_months_with_oos>0:
        months_with_properly_using_oos_perc = \
    round(100*(num_of_months_with_properly_using_oos/num_of_months_with_oos), 2)
    else:
        return np.nan
    
    return months_with_properly_using_oos_perc

In [4]:
export = pd.DataFrame(index = [x.split('_wo_151617')[0] for x in data_sets],\
                     columns = ['had_online_orders_before',\
                               'had_online_orders_last_month',\
                               'had_online_orders_before_and_didnt_last_month',\
                               'had_properly_used_online_orders_before',\
                               'had_properly_used_online_orders_last_month',\
                               'had_properly_used_online_orders_before_and_didnt_last_month',\
                               'had_online_orders_but_stopped_properly_using_them',\
                               'months_with_properly_using_online_orders_on_average_%'])

In [5]:
for filename in [x.split('_wo_151617')[0] for x in data_sets]:
    df = \
    bucket.load_csv_from_s3(file_name='churn_analysis_based_on_behaviour/data/'+date_dir+'/'+filename+'_wo_151617.csv')

    had_online_orders_before = df[df['had_online_orders_before']>0]['spot_id'].unique()
    had_online_orders_last_month = df[df['had_online_orders_last_month']>0]['spot_id'].unique()
    had_online_orders_before_and_didnt_last_month = df[df['had_online_orders_before_and_didnt_last_month']>0]['spot_id'].unique()

    had_properly_used_online_orders_before = \
    df[df['had_properly_used_online_orders_inquiries_before']>0]['spot_id'].unique()
    had_properly_used_online_orders_last_month = \
    df[df['had_properly_used_online_orders_inquiries_last_month']>0]['spot_id'].unique()
    had_properly_used_online_orders_before_and_didnt_last_month = \
    df[df['had_properly_used_online_orders_inquiries_before_and_didnt_last_month']>0]['spot_id'].unique()

    had_online_orders_but_stopped_properly_using_them = \
    df[(df['had_properly_used_online_orders_inquiries_before_and_didnt_last_month']==1)&\
      (df['had_online_orders_last_month']==1)]['spot_id'].unique()

    export.loc[filename, 'had_online_orders_before'] = len(had_online_orders_before)
    export.loc[filename, 'had_online_orders_last_month'] = len(had_online_orders_last_month)
    export.loc[filename, 'had_online_orders_before_and_didnt_last_month'] = \
    (len(had_online_orders_before_and_didnt_last_month), round(100*(len(had_online_orders_before_and_didnt_last_month)/len(had_online_orders_before)), 2))

    export.loc[filename, 'had_properly_used_online_orders_before'] = \
    (len(had_properly_used_online_orders_before), round(100*(len(had_properly_used_online_orders_before)/len(had_online_orders_before)), 2))
    export.loc[filename, 'had_properly_used_online_orders_last_month'] = \
    (len(had_properly_used_online_orders_last_month), round(100*(len(had_properly_used_online_orders_last_month)/len(had_online_orders_last_month)), 2))
    export.loc[filename, 'had_properly_used_online_orders_before_and_didnt_last_month'] = \
    (len(had_properly_used_online_orders_before_and_didnt_last_month), round(100*(len(had_properly_used_online_orders_before_and_didnt_last_month)/len(had_properly_used_online_orders_before)), 2))

    export.loc[filename, 'had_online_orders_but_stopped_properly_using_them'] = \
    (len(had_online_orders_but_stopped_properly_using_them), \
         round(100*(len(had_online_orders_but_stopped_properly_using_them)/len(had_properly_used_online_orders_before_and_didnt_last_month)), 2))
    
    df['months_with_properly_using_online_orders_perc'] = \
    df.groupby('spot_id').apply(lambda x: get_months_with_properly_using_oos_perc(spot = \
                                                                                 x[['had_online_orders_last_month',\
                                                                                   'had_properly_used_online_orders_inquiries_last_month']]))

    export.loc[filename, 'months_with_properly_using_online_orders_on_average_%'] = \
    round(df[df['months_with_properly_using_online_orders_perc'].notnull()][['spot_id','months_with_properly_using_online_orders_perc']].\
    drop_duplicates()['months_with_properly_using_online_orders_perc'].mean(), 2)

In [8]:
export.transpose().to_csv('data/'+date_dir+'/exports/had_OOs_vs_properly_used_OOs.csv')